# Plantilla: Proyecto
## Tema 2: Clasificación de barrios en función de su tipología.
Documento plantilla para el proyecto con los imports de las librerias más importantes.

In [31]:
import nltk                      # Natural Language Toolkit
import scrapy                    # Web scraping   
import requests                  # HTTP requests
import selenium                  # Web automation
import numpy as np               # Numerical computing
import pandas as pd              # Data manipulation
import seaborn as sns            # Data visualization
import tensorflow as tf          # Machine learning
import matplotlib.pyplot as plt  # Data visualization
import os                        # Operating system

!pip install openpyxl
import openpyxl                  # Excel files
import zipfile                   # Zip files

# Automatización en la recogida de variables (dataset Barris)

Primero crearemos el diccionario `barrios_dict` donde se haga la asignación entre los nombres de los Barrios y el par distrito-barrio.

In [32]:
idBarrios = pd.read_csv('./data/barrios/barrios_codigos.csv', delimiter=';')
idBarrios = idBarrios[['Nombre', 'Codigo distrito', 'Codigo barrio']]

barrios_dict = {}

for index, row in idBarrios.iterrows():
    nombre = row['Nombre']
    codigo_distrito = row['Codigo distrito']
    codigo_barrio = row['Codigo barrio']
    
    barrio_dict = {
        'Codigo distrito': codigo_distrito,
        'Codigo barrio': codigo_barrio
    }
    
    barrios_dict[nombre] = barrio_dict

Definimos la función `get_barrio_name` que nos permite realizar la asignación distrito-barrio -> nombre del barrio.

In [33]:
def get_barrio_name(codigo_distrito, codigo_barrio):
    for nombre, barrio_dict in barrios_dict.items():
        if barrio_dict['Codigo distrito'] == codigo_distrito and barrio_dict['Codigo barrio'] == codigo_barrio:
            return nombre

Definimos a continuación unas funciones que nos ayudarán a extraer correctamente las tablas de las variables deseadas de los distintos excel.

In [34]:
def find_string_in_df(df, string):
    for i in range(len(df)):
        if string in str(df.iloc[i, 0]):
            return i
    return None

def find_skip(df, row):
    # Encontrar la primera fila que no esté completamente llena de NAs después de la fila especificada
    for i in range(row + 2, len(df)): # el +2 en lugar de +1 es para poder ponerlo en castellano xD
        if not df.iloc[i].isna().all():
            return i - row

    return 0  # Si todas las filas después de 'row' están completamente llenas de NAs, skip es 0

def extract_df(df, row, skip, n=None):
    if n is None:
        # Encontrar n dinámicamente.
        last_row = row + skip
        while last_row < len(df) and not df.iloc[last_row].isna().all():
            last_row += 1
        n = last_row - row - skip
    extracted_df = df.iloc[row + skip:row + skip + n, :]
    
    # Verificar si TODAS las filas en cada columna son NA
    columns_to_drop = extracted_df.columns[extracted_df.isna().all()]
    
    # Eliminar solo las columnas donde todas las filas son NA
    extracted_df = extracted_df.drop(columns_to_drop, axis=1)
    
    return extracted_df

def extract_df_by_string(df, string, n=None):
    row = find_string_in_df(df, string)
    skip = find_skip(df, row)
    return extract_df(df, row, skip, n)

In [35]:
#dataframes = { 'dfSup': dfSup,
#                    'dfPobSexEdad': dfPobSexEdad,
#                    'dfPobSexLugar': dfPobSexLugar,
#                    'dfPobExtCont': dfPobExtCont,
#                    'dfHojasFamComp': dfHojasFamComp,
#                    'dfIndicadoresDemograficosPadron': dfIndicadoresDemograficosPadron,
#                    'dfMovPadron': dfMovPadron,
#                    'dfIndicadoresDemograficosMovPadron': dfIndicadoresDemograficosMovPadron,
#                    'dfVehicTipo': dfVehicTipo,
#                    'dfTurismPot': dfTurismPot,
#                    'dfBienesInmAno': dfBienesInmAno,
#                    'dfBienesInmSup': dfBienesInmSup,
#                    'dfBienesInmVal': dfBienesInmVal,
#                    'dfBienesValMedio': dfBienesValMedio,
#                    'dfSupAparc': dfSupAparc,
#                    'dfElecGen': dfElecGen
#                 }

Se definen a continuación funciones para hacer un preprocesamiento de los diferentes datos que se adquirirán.

In [36]:
def clean_dataframe(df):
    # Eliminar filas y columnas con todos los valores NaN
    df = df.dropna(axis=0, how='all').dropna(axis=1, how='all')
    # Establecer la primera fila como nombres de columnas y reindexar
    df.columns = df.iloc[0]
    df = df.iloc[1:].reset_index(drop=True)
    return df

def modify_first_column_name(df, column_name):
    df.columns = [column_name] + list(df.columns[1:])
    return df
        
def extract_dataframes_from_excel(excel_path):
    dataframes = {}
    sheets = {
        'Padrón': ['Superficie y densidad de población', 'Población por sexo y edad (grandes grupos)',
                   'Población según lugar de nacimiento y sexo', 'Población extranjera según nacionalidad, por contienentes',
                   'Número de hojas familiares según composición', 'Indicadores demográficos'],
        'MovPadrón': ['Resumen de movimientos registrados en el Padrón Municipal', 'Indicadores demográficos'],
        'Vehículos': ['Vehículos según tipo', 'Turismos según potencia'],
        'Catastro': ['Bienes Inmuebles según el año de antigüedad',
                     'Bienes Inmuebles construidas después de 1800 según superficie construida',
                     'Bienes Inmuebles construidas después de 1800 según valor catastral',
                     'Valores catastrales medios', 'Superficie total de los aparcamientos'],
        'Elecciones': ['Votos a candidaturas']
    }
    
    for sheet, strings in sheets.items():
        df_sheet = pd.read_excel(excel_path, sheet_name=sheet)
        for string in strings:
            df = extract_df_by_string(df_sheet, string)
            if df is not None:
                df = clean_dataframe(df)
                if sheet == 'Padrón' and string in ['Población por sexo y edad (grandes grupos)', 'Población según lugar de nacimiento y sexo', 'Población extranjera según nacionalidad, por contienentes']:
                    modify_first_column_name(df, 'Género')
                elif sheet == 'Vehículos' and string == 'Vehículos según tipo':
                    modify_first_column_name(df, 'Tipo de vehículo')
                elif sheet == "Vehículos" and string == "Turismos según potencia":
                    modify_first_column_name(df, 'Potencia')
                elif sheet == 'Elecciones' and string == 'Votos a candidaturas':
                    modify_first_column_name(df, 'Partido')
                dataframes[f'{sheet}_{string.replace(" ", "_")}'] = df
    
    return dataframes


Esta función permite sacar el nombre del barrio infiriéndolo por el nombre del Excel (sigue la estructura de `Distrito_{num_distrito}_Barrio_{num_barrio}.xlsx`).

In [37]:
def get_barrio_name_from_filename(filename, barrios_dict):
    # Extraer el nombre del distrito y el número de barrio del nombre del archivo
    distrito_numero, barrio_numero = os.path.splitext(filename)[0].split('_')[1:4][::2]
    
    # Buscar el nombre del distrito y el número de barrio en barrios_dict
    for nombre, info_barrio in barrios_dict.items():
        if info_barrio['Codigo distrito'] == int(distrito_numero) and info_barrio['Codigo barrio'] == int(barrio_numero):
            return nombre
    
    return None

Se definen a continuación funciones que nos permitirán realizar el proceso de creación de la carpeta para un año concreto, descarga del zip correspondiente y su descompresión automática.

In [38]:
def download_zip(year):
    url = f'https://www.valencia.es/estadistica/inf_dtba/{year}/Barrios{year}.zip'
    response = requests.get(url)
    with open(f'./data/barrios/{year}/{year}.zip', 'wb') as file:
        file.write(response.content)

def create_folder(year):
    folder_path = f'./data/barrios/{year}'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
def unzip_file(year):
    with zipfile.ZipFile(f'./data/barrios/{year}/{year}.zip', 'r') as zip_ref:
        zip_ref.extractall(f'./data/barrios/{year}')
        
def download_and_unzip(year):
    create_folder(year)
    download_zip(year)
    unzip_file(year)
    

Finalmente, la función `get_data` permite recibir una lista de años y devuelve los datos correctamente preprocesados ya para las variables que nos interesan.

In [39]:
def get_data(years):
    datos = {}
    for year in years:
        download_and_unzip(year)
        datos[year] = {}
        for filename in os.listdir(f'./data/barrios/{year}'):
            if filename.endswith('.xlsx'):
                excel_path = os.path.join(f'./data/barrios/{year}', filename)
                dataframes = extract_dataframes_from_excel(excel_path)
                barrio_name = get_barrio_name_from_filename(filename, barrios_dict)
                datos[year][barrio_name] = dataframes
    return datos

In [40]:
# Probar con lista de [2023, 2024] como year

years = [2023, 2024]

datosBarris = get_data(years)

In [43]:
datosBarris[2023]['BENICALAP']['Padrón_Población_por_sexo_y_edad_(grandes_grupos)']

,Género,Total,0-15,16-64,65 y más
0,Total,42725,6502,28408,7815
1,Hombres,20663,3336,13950,3377
2,Mujeres,22062,3166,14458,4438
